# __Poisson matrix tests__

## Outline
- __Dependencies__
- __Dynamic ansatz__
    - Noiseless
    - With noise using Qiskit AER
- __Static ansatz__
    - Noiseless
    - With noise using Qiskit AER

In [9]:
import numpy as np
from device_var_lse_solver import DeviceVarLSESolver
from device import DeviceType
from conditioned_matrix import poissonMatrix
from non_dynamical_ansatz import fixed_layered_ansatz
from tqdm import tqdm
from device import Device

qubits = 4
b = np.ones(2**qubits)/np.sqrt(2**qubits)
depth=10

assert qubits % 2 == 0 # Needed for Poisson matrix

In [10]:
# As defined by [10]
def TRC_ADA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += d*z
    return trc

def TRC_ASA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += z*depth
    return trc

## __Dynamic ansatz__

In [11]:
# For poisson matrix
N_runs = 10
N_steps = 1500
redo_calc = 5

local_noiseless_dynamic_trc = np.zeros((redo_calc, ))
global_noiseless_dynamic_trc = np.zeros((redo_calc, ))
local_noise_dynamic_trc = np.zeros((redo_calc, ))
global_noise_dynamic_trc = np.zeros((redo_calc, ))

local_noiseless_static_trc = np.zeros((redo_calc, ))
global_noiseless_static_trc = np.zeros((redo_calc, ))
local_noise_static_trc = np.zeros((redo_calc, ))
global_noise_static_trc = np.zeros((redo_calc, ))

local_noise_dynamic_error = np.zeros((redo_calc, ))
local_noiseless_dynamic_error = np.zeros((redo_calc, ))
local_noise_static_error = np.zeros((redo_calc, ))
local_noiseless_static_error = np.zeros((redo_calc, ))

global_noise_dynamic_error = np.zeros((redo_calc, ))
global_noiseless_dynamic_error = np.zeros((redo_calc, ))
global_noise_static_error = np.zeros((redo_calc, ))
global_noiseless_static_error = np.zeros((redo_calc, ))

local_noise_dynamic_solution = np.zeros((redo_calc, 2**qubits))
local_noiseless_dynamic_solution = np.zeros((redo_calc, 2**qubits))
local_noise_static_solution = np.zeros((redo_calc, 2**qubits))
local_noiseless_static_solution = np.zeros((redo_calc, 2**qubits))

global_noise_dynamic_solution = np.zeros((redo_calc, 2**qubits))
global_noiseless_dynamic_solution = np.zeros((redo_calc, 2**qubits))
global_noise_static_solution = np.zeros((redo_calc, 2**qubits))
global_noiseless_static_solution = np.zeros((redo_calc, 2**qubits))

correct_solutions = np.zeros((redo_calc, 2**qubits))


device = Device(DeviceType.QISKIT_AER, qubits=qubits)

param_shape = (qubits + depth*(qubits+qubits -2),)

pbar = tqdm(range(redo_calc))
for i in pbar:
        a = poissonMatrix(2**(qubits//2)).todense()
        classical_solution = np.linalg.solve(a, b)
        normalized_classical_solution = np.square(classical_solution / np.linalg.norm(classical_solution))
        normalized_classical_solution /= np.linalg.norm(normalized_classical_solution)

        correct_solutions[i] = np.abs(normalized_classical_solution)

        ###################################
        #####    DYNAMIC NOISELESS    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=True, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=False)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=False, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)

        pbar.set_description("Local Dynamic Noiseless (1/8)")
        solution_local, param_local, it_count_local = lse_local.solve()
        pbar.set_description("Global Dynamic Noiseless (2/8)")
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noiseless_dynamic_solution[i] = solution_local
        global_noiseless_dynamic_solution[i] = solution_global

        local_noiseless_dynamic_trc[i] = TRC_ADA(it_count_local)
        global_noiseless_dynamic_trc[i] = TRC_ADA(it_count_global)

        local_noiseless_dynamic_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noiseless_dynamic_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

        ###################################
        #####    DYNAMIC NOISEFULL    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=True, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=False, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)

        pbar.set_description("Local Dynamic Noise (3/8)")
        solution_local, param_local, it_count_local = lse_local.solve()
        pbar.set_description("Global Dynamic Noise (4/8)")
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noise_dynamic_solution[i] = solution_local
        global_noise_dynamic_solution[i] = solution_global

        local_noise_dynamic_trc[i] = TRC_ADA(it_count_local)
        global_noise_dynamic_trc[i] = TRC_ADA(it_count_global)

        local_noise_dynamic_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noise_dynamic_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

        ###################################
        #####    STATIC  NOISELESS    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                local=True, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                local=False, 
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                silent=True)

        pbar.set_description("Local Static Noiseless (5/8)")
        solution_local, param_local, it_count_local = lse_local.solve()
        pbar.set_description("Global Static Noiseless (6/8)")
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noiseless_static_solution[i] = solution_local
        global_noiseless_static_solution[i] = solution_global

        local_noiseless_static_trc[i] = TRC_ASA(it_count_local)
        global_noiseless_static_trc[i] = TRC_ASA(it_count_global)

        local_noiseless_static_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noiseless_static_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

        ###################################
        #####    STATIC  NOISEFULL    #####
        ###################################
        lse_local = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=True, 
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)
        
        lse_global = DeviceVarLSESolver(a,
                b, 
                method='direct',
                local=False, 
                ansatz=fixed_layered_ansatz,
                weights=param_shape,
                lr=0.1, 
                steps=N_steps,
                threshold=0.001, 
                epochs=N_runs,
                device=device,
                silent=True)

        pbar.set_description("Local Static Noise (7/8)")
        solution_local, param_local, it_count_local = lse_local.solve()
        pbar.set_description("Global Static Noise (8/8)")
        solution_global, param_global, it_count_global = lse_global.solve()
        solution_local /= np.linalg.norm(solution_local)
        solution_global /= np.linalg.norm(solution_global)

        local_noise_static_solution[i] = solution_local
        global_noise_static_solution[i] = solution_global

        local_noise_static_trc[i] = TRC_ASA(it_count_local)
        global_noise_static_trc[i] = TRC_ASA(it_count_global)

        local_noise_static_error[i] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
        global_noise_static_error[i] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

Local Dynamic Noiseless (1/8):   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/10:  38%|███▊      | 576/1500 [00:08<00:14, 64.85it/s, best loss=0.167, last improvement in step=78, loss=0.167]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 2/10:  41%|████      | 608/1500 [00:19<00:28, 31.37it/s, best loss=0.118, last improvement in step=108, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 3/10:  59%|█████▉    | 882/1500 [00:41<00:28, 21.37it/s, best loss=0.0638, last improvement in step=383, loss=0.0637]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 4/10:  64%|██████▎   | 956/1500 [00:55<00:31, 17.19it/s, best loss=0.0639, last improvement in step=456, loss=0.064]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 5/10:  46%|████▌     | 688/1500 [00:45<00:53, 15.21it/s, best loss=0.0638, last improvement in step=188, loss=0.0641]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 6/10:  45%|████▌     | 676/1500 [00:48<00:59,

Global Dynamic Noise (4/8):   0%|          | 0/5 [25:23<?, ?it/s]    c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Local Dynamic Noiseless (1/8):  20%|██        | 1/5 [39:06<2:36:25, 2346.28s/it]

Epoch 1/10:  42%|████▏     | 623/1500 [00:10<00:15, 57.65it/s, best loss=0.167, last improvement in step=124, loss=0.167]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 2/10:  41%|████      | 615/1500 [00:19<00:28, 31.14it/s, best loss=0.118, last improvement in step=116, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 3/10:  50%|████▉     | 743/1500 [00:34<00:35, 21.47it/s, best loss=0.0637, last improvement in step=244, loss=0.064]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 4/10:  41%|████      | 615/1500 [00:38<00:55, 16.03it/s, best loss=0.0638, last improvement in step=116, loss=0.0645]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 5/10:  41%|████      | 618/1500 [00:47<01:07, 13.13it/s, best loss=0.0638, last improvement in step=118, loss=0.0638]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 6/10:  46%|████▋     | 696/1500 [01:03<01:13

Global Dynamic Noise (4/8):  20%|██        | 1/5 [1:06:04<2:36:25, 2346.28s/it]  c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Local Dynamic Noiseless (1/8):  40%|████      | 2/5 [1:16:40<1:54:35, 2291.87s/it]

...

Global Dynamic Noise (4/8):  40%|████      | 2/5 [1:37:56<1:54:35, 2291.87s/it]    c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Local Dynamic Noiseless (1/8):  60%|██████    | 3/5 [1:49:13<1:11:15, 2137.54s/it]

Epoch 1/10:  38%|███▊      | 573/1500 [00:08<00:13, 69.58it/s, best loss=0.167, last improvement in step=76, loss=0.167]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 2/10:  40%|████      | 607/1500 [00:16<00:23, 37.44it/s, best loss=0.118, last improvement in step=108, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 3/10:  40%|████      | 606/1500 [00:23<00:34, 26.14it/s, best loss=0.0638, last improvement in step=107, loss=0.0637]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 4/10:  44%|████▍     | 667/1500 [00:32<00:41, 20.28it/s, best loss=0.0638, last improvement in step=167, loss=0.0638]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 5/10:  55%|█████▌    | 832/1500 [00:51<00:41, 16.26it/s, best loss=0.0638, last improvement in step=332, loss=0.0647]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 6/10:  53%|█████▎    | 788/1500 [00:56<00:51

Global Dynamic Noise (4/8):  60%|██████    | 3/5 [2:17:17<1:11:15, 2137.54s/it]    c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Local Dynamic Noiseless (1/8):  80%|████████  | 4/5 [2:30:54<38:00, 2280.84s/it]  

Epoch 1/10:  39%|███▉      | 583/1500 [00:10<00:16, 55.83it/s, best loss=0.167, last improvement in step=87, loss=0.167]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 2/10:  40%|███▉      | 598/1500 [00:19<00:29, 30.75it/s, best loss=0.118, last improvement in step=99, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 3/10:  42%|████▏     | 623/1500 [00:29<00:40, 21.45it/s, best loss=0.118, last improvement in step=124, loss=0.118]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 4/10:  41%|████      | 614/1500 [00:37<00:54, 16.40it/s, best loss=0.0638, last improvement in step=115, loss=0.07]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 5/10:  42%|████▏     | 634/1500 [00:48<01:06, 13.00it/s, best loss=0.0638, last improvement in step=135, loss=0.0643]
Loss has not improved in last 500 steps.
Increasing circuit depth.
Epoch 6/10:  43%|████▎     | 649/1500 [00:58<01:17, 11.

Global Dynamic Noise (4/8):  80%|████████  | 4/5 [2:58:55<38:00, 2280.84s/it]    c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')
Global Static Noise (8/8): 100%|██████████| 5/5 [3:11:56<00:00, 2303.36s/it]    


## __Save vals__

In [12]:
## Save the vals
filename = f"data/result_poisson_test_2"
saved_dict = {
    # Noise dynamic
    "local_noise_dynamic_trc" : local_noise_dynamic_trc,
    "local_noise_dynamic_error" : local_noise_dynamic_error,
    "local_noise_dynamic_solution" : local_noise_dynamic_solution,
    "global_noise_dynamic_trc" : global_noise_dynamic_trc,
    "global_noise_dynamic_error" : global_noise_dynamic_error,
    "global_noise_dynamic_solution" : global_noise_dynamic_solution,

    # Noiseless dynamic
    "local_noiseless_dynamic_trc" : local_noiseless_dynamic_trc,
    "local_noiseless_dynamic_error" : local_noiseless_dynamic_error,
    "local_noiseless_dynamic_solution" : local_noiseless_dynamic_solution,
    "global_noiseless_dynamic_trc" : global_noiseless_dynamic_trc,
    "global_noiseless_dynamic_error" : global_noiseless_dynamic_error,
    "global_noiseless_dynamic_solution" : global_noiseless_dynamic_solution,

    # Noise static
    "local_noise_static_trc" : local_noise_static_trc,
    "local_noise_static_error" : local_noise_static_error,
    "local_noise_static_solution" : local_noise_static_solution,
    "global_noise_static_trc" : global_noise_static_trc,
    "global_noise_static_error" : global_noise_static_error,
    "global_noise_static_solution" : global_noise_static_solution,

    # Noiseless static
    "local_noiseless_static_trc" : local_noiseless_static_trc,
    "local_noiseless_static_error" : local_noiseless_static_error,
    "local_noiseless_static_solution" : local_noiseless_static_solution,
    "global_noiseless_static_trc" : global_noiseless_static_trc,
    "global_noiseless_static_error" : global_noiseless_static_error,
    "global_noiseless_static_solution" : global_noiseless_static_solution,

    "normalized_classical_solution" : normalized_classical_solution
}
np.savez(filename, **saved_dict)
print(f"Saved {filename}")

Saved data/result_poisson_test_2


In [13]:
data = np.load(f"{filename}.npz")

local_noise_dynamic_error = data["local_noise_dynamic_error"]
print(local_noise_dynamic_error)

[0.31248945 0.21834818 0.27773364 0.2703654  0.24569366]
